## ⚠️ BEFORE YOU START

**What day is it today?**

- ✅ **Friday after 4pm ET:** Perfect! Run all cells below
- ❌ **Monday-Thursday:** STOP! Wait until Friday after market close
- ❌ **Friday before 4pm:** Wait until market closes at 4pm ET

**Why Friday close?** GHB Strategy uses weekly closing prices. Trading mid-week gives false signals and won't match backtested performance.

---

# GHB Strategy Portfolio Scanner
**Gold-Gray-Blue Weekly Trading System**

## 📅 WEEKLY TRADING SCHEDULE

⚠️ **IMPORTANT:** Only run this scanner on **FRIDAY after 4pm ET** (after market close)

### Your Weekly Routine:

**FRIDAY (After 4pm ET)**
- 📊 Run this notebook (all cells)
- 📋 Review signals: BUY (P1), HOLD (P2/N1), SELL (N2)
- 📝 Make your trade list for Monday
- ⏱️ Time: 10-15 minutes

**WEEKEND (Saturday/Sunday)**
- 💭 Review and confirm your plan
- 🧮 Calculate position sizes (8-10% each)
- ✅ Prepare for Monday execution

**MONDAY (Market Open - 9:30am ET)**
- 🔵 **FIRST:** Execute ALL sell signals (N2 stocks)
- 🟡 **THEN:** Enter new buy positions (P1 stocks - top 3-5)
- ⏱️ Time: 15-30 minutes

---

**Last Run:** {current_date}  
**Strategy:** GHB Strategy (Gold-Gray-Blue)  
**Universe:** 25 Optimized Stocks (Your Watchlist + Top Performers)  
**Expected Annual Return:** +514%

In [1]:
# Import Required Libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded successfully
📅 Current Date: 2026-01-15


## 1. Define Stock Universe

The 25 stocks selected for optimal performance:
- Your 12-stock watchlist (core holdings)
- 13 top performers from backtesting (2021-2025)
- Expected: +514% annual returns, ~14 trades/year

This delivers 117% of full universe returns with much easier management!

In [2]:
# GHB Strategy Optimized Portfolio - 25 Stocks
# Core: Your 12-stock watchlist + Top 13 performers from backtesting
GHB_UNIVERSE = [
    'ALAB', 'AMAT', 'AMD', 'ARM', 'ASML', 'AVGO', 'BKNG', 'CEG', 
    'COST', 'DASH', 'FANG', 'FTNT', 'GOOG', 'GOOGL', 'META', 'MRNA',
    'MRVL', 'MSFT', 'MU', 'NFLX', 'NVDA', 'PANW', 'PLTR', 'TSLA', 'TSM'
]

print(f"📊 Universe: {len(GHB_UNIVERSE)} stocks (Optimized Portfolio)")
print(f"📈 Stocks: {', '.join(sorted(GHB_UNIVERSE[:10]))}...")
print(f"💡 Expected: +514% annual return, ~14 trades/year")

📊 Universe: 25 stocks (Optimized Portfolio)
📈 Stocks: ALAB, AMAT, AMD, ARM, ASML, AVGO, BKNG, CEG, COST, DASH...
💡 Expected: +514% annual return, ~14 trades/year


## 2. Calculate Weekly Larsson States

For each stock, calculate:
- **Weekly Close** (Friday)
- **200-Day SMA** (D200)
- **4-Week ROC** (Rate of Change)
- **Weekly State** (P1/P2/N1/N2)

In [3]:
def calculate_weekly_larsson_state(ticker):
    """
    Calculate weekly Larsson state for a ticker
    Returns: dict with ticker data or None if error
    """
    try:
        # Download 1 year of daily data
        stock = yf.Ticker(ticker)
        df = stock.history(period='1y', interval='1d')
        
        if df.empty or len(df) < 200:
            return None
        
        # Get latest close
        close = df['Close'].iloc[-1]
        
        # Calculate 200-day SMA
        d200 = df['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate 4-week ROC (20 trading days)
        if len(df) >= 20:
            price_4w_ago = df['Close'].iloc[-20]
            roc_4w = ((close - price_4w_ago) / price_4w_ago) * 100
        else:
            roc_4w = 0
        
        # Calculate distance from D200
        distance_pct = ((close - d200) / d200) * 100
        
        # Determine Weekly Larsson state (Strategy D rules)
        if close > d200:
            # Price above D200
            if roc_4w > 5 or distance_pct > 10:
                state = 'P1'  # Strong bullish
                signal = '🟡 BUY'
            else:
                state = 'P2'  # Consolidation
                signal = '⚪ HOLD'
        else:
            # Price below D200
            if distance_pct > -5:
                state = 'N1'  # Shallow pullback
                signal = '⚪ HOLD'
            else:
                state = 'N2'  # Downtrend
                signal = '🔵 SELL'
        
        return {
            'Ticker': ticker,
            'Close': close,
            'D200': d200,
            'Distance_%': distance_pct,
            'ROC_4W_%': roc_4w,
            'State': state,
            'Signal': signal
        }
        
    except Exception as e:
        print(f"❌ Error processing {ticker}: {str(e)}")
        return None

print("✅ Calculation function defined")

✅ Calculation function defined


## 3. Scan All 39 Stocks

This will take 1-2 minutes to download data and calculate states...

In [4]:
print(f"🔄 Scanning {len(GHB_UNIVERSE)} stocks... Please wait...\n")

results = []
for i, ticker in enumerate(GHB_UNIVERSE, 1):
    print(f"  [{i:2d}/{len(GHB_UNIVERSE)}] Processing {ticker:6s}...", end='\r')
    result = calculate_weekly_larsson_state(ticker)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)

print(f"\n✅ Scan complete! Processed {len(df_results)}/{len(GHB_UNIVERSE)} stocks")
print(f"❌ Failed: {len(GHB_UNIVERSE) - len(df_results)} stocks")

🔄 Scanning 25 stocks... Please wait...

  [25/25] Processing TSM   ...
✅ Scan complete! Processed 25/25 stocks
❌ Failed: 0 stocks


## 4. Strategy D Signals

### Buy Signals (P1 - Gold)
**Action:** Enter new positions or add to existing  
**Requirements:** Price > D200 + Strong momentum

In [5]:
# Filter P1 (BUY) signals
p1_signals = df_results[df_results['State'] == 'P1'].sort_values('ROC_4W_%', ascending=False)

print("=" * 100)
print(f"🟡 P1 (GOLD) - BUY SIGNALS: {len(p1_signals)} stocks")
print("=" * 100)

if len(p1_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Above D200':<12} {'4W ROC':<10} {'Strength':<15}")
    print("-" * 100)
    
    for _, row in p1_signals.iterrows():
        # Determine strength
        if row['Distance_%'] > 30:
            strength = "🔥 EXPLOSIVE"
        elif row['Distance_%'] > 20:
            strength = "💪 VERY STRONG"
        elif row['ROC_4W_%'] < 0:
            strength = "✅ PULLBACK BUY"
        else:
            strength = "✅ STRONG"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {strength:<15}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - Enter NEW positions in these P1 stocks")
    print("   - Add to existing positions if capital available")
    print("   - Prioritize: Explosive > Very Strong > Strong > Pullback Buy")
else:
    print("\n⚠️  No P1 buy signals this week")
    print("   - Wait for new P1 entries")
    print("   - Hold existing positions")

🟡 P1 (GOLD) - BUY SIGNALS: 14 stocks

Ticker   Price      D200       Above D200   4W ROC     Strength       
----------------------------------------------------------------------------------------------------
MU       $333.35    $158.63        +110.1%   +43.43% 🔥 EXPLOSIVE    
MRNA     $40.58     $27.51          +47.5%   +35.76% 🔥 EXPLOSIVE    
ALAB     $172.14    $138.53         +24.3%   +18.77% 💪 VERY STRONG  
ASML     $1263.72   $864.90         +46.1%   +17.44% 🔥 EXPLOSIVE    
AMAT     $301.89    $197.21         +53.1%   +16.63% 🔥 EXPLOSIVE    
TSM      $327.11    $243.89         +34.1%   +14.03% 🔥 EXPLOSIVE    
COST     $950.98    $948.70          +0.2%   +10.53% ✅ STRONG       
GOOGL    $335.84    $224.35         +49.7%    +9.55% 🔥 EXPLOSIVE    
GOOG     $336.31    $225.34         +49.2%    +9.29% 🔥 EXPLOSIVE    
AMD      $223.60    $167.42         +33.6%    +6.90% 🔥 EXPLOSIVE    
NVDA     $183.14    $163.27         +12.2%    +3.05% ✅ STRONG       
AVGO     $339.89    $295.36    

### Hold Signals (P2 & N1 - Gray)
**Action:** Continue holding existing positions  
**Meaning:** Normal consolidation/pullback in trend

In [8]:
# Filter P2 and N1 (HOLD) signals
hold_signals = df_results[df_results['State'].isin(['P2', 'N1'])].sort_values('Distance_%', ascending=False)

print("=" * 100)
print(f"⚪ P2/N1 (GRAY) - HOLD SIGNALS: {len(hold_signals)} stocks")
print("=" * 100)

if len(hold_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Distance':<12} {'4W ROC':<10} {'State':<8} {'Status':<20}")
    print("-" * 100)
    
    for _, row in hold_signals.iterrows():
        if row['State'] == 'P2':
            status = "Consolidation"
        else:
            status = "Shallow Pullback"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {row['State']:<8} {status:<20}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - HOLD all existing positions")
    print("   - Do NOT sell - this is normal consolidation")
    print("   - Watch for transition to P1 (upgrade) or N2 (downgrade)")
else:
    print("\n✅ No stocks in consolidation phase")

⚪ P2/N1 (GRAY) - HOLD SIGNALS: 6 stocks

Ticker   Price      D200       Distance     4W ROC     State    Status              
----------------------------------------------------------------------------------------------------
FANG     $153.73    $141.78          +8.4%    +3.13% P2       Consolidation       
MRVL     $81.21     $74.95           +8.4%    -3.33% P2       Consolidation       
CEG      $330.38    $317.51          +4.1%    -9.64% P2       Consolidation       
PANW     $190.93    $192.69          -0.9%    +2.05% N1       Shallow Pullback    
BKNG     $5187.02   $5268.25         -1.5%    -4.60% N1       Shallow Pullback    
MSFT     $459.38    $480.24          -4.3%    -3.57% N1       Shallow Pullback    

💡 RECOMMENDATION:
   - HOLD all existing positions
   - Do NOT sell - this is normal consolidation
   - Watch for transition to P1 (upgrade) or N2 (downgrade)


### Sell Signals (N2 - Blue)
**Action:** Exit positions immediately  
**Requirements:** Price < D200 + Weak momentum  
**⚠️ CRITICAL: Execute these sells on Monday!**

In [6]:
# Filter N2 (SELL) signals
n2_signals = df_results[df_results['State'] == 'N2'].sort_values('Distance_%', ascending=True)

print("=" * 100)
print(f"🔵 N2 (BLUE) - SELL SIGNALS: {len(n2_signals)} stocks")
print("=" * 100)

if len(n2_signals) > 0:
    print(f"\n⚠️  EXIT THESE POSITIONS ON MONDAY!\n")
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Below D200':<12} {'4W ROC':<10} {'Severity':<15}")
    print("-" * 100)
    
    for _, row in n2_signals.iterrows():
        # Determine severity
        if row['Distance_%'] < -20:
            severity = "🚨 SEVERE"
        elif row['Distance_%'] < -10:
            severity = "⚠️  MAJOR"
        else:
            severity = "📉 MINOR"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {severity:<15}")
    
    print("\n💡 ACTION REQUIRED:")
    print("   - SELL all N2 positions on Monday at market open")
    print("   - Do NOT wait for bounce - trend is broken")
    print("   - Preserve capital for new P1 opportunities")
else:
    print("\n✅ No sell signals - all positions healthy!")

🔵 N2 (BLUE) - SELL SIGNALS: 5 stocks

⚠️  EXIT THESE POSITIONS ON MONDAY!

Ticker   Price      D200       Below D200   4W ROC     Severity       
----------------------------------------------------------------------------------------------------
ARM      $104.99    $137.20         -23.5%   -13.30% 🚨 SEVERE       
NFLX     $88.55     $113.13         -21.7%    -6.37% 🚨 SEVERE       
FTNT     $76.39     $90.38          -15.5%    -7.37% ⚠️  MAJOR      
META     $615.52    $674.43          -8.7%    -6.33% 📉 MINOR        
DASH     $213.13    $228.38          -6.7%    -6.50% 📉 MINOR        

💡 ACTION REQUIRED:
   - SELL all N2 positions on Monday at market open
   - Do NOT wait for bounce - trend is broken
   - Preserve capital for new P1 opportunities


## 5. Weekly Summary

Quick overview of portfolio status and action items

In [10]:
print("=" * 100)
print("📊 GHB STRATEGY WEEKLY SUMMARY")
print("=" * 100)

print(f"\n🟡 BUY Signals (P1):  {len(p1_signals)} stocks")
print(f"⚪ HOLD Signals (P2/N1): {len(hold_signals)} stocks")
print(f"🔵 SELL Signals (N2): {len(n2_signals)} stocks")
print(f"📊 Total Scanned: {len(df_results)}/{len(GHB_UNIVERSE)} stocks")

# Calculate portfolio health
total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)

pct_bullish = (total_bullish / total * 100) if total > 0 else 0
pct_bearish = (total_bearish / total * 100) if total > 0 else 0

print(f"\n📈 Market Health:")
print(f"   Bullish: {pct_bullish:.1f}% ({total_bullish} stocks)")
print(f"   Neutral: {(total_neutral/total*100):.1f}% ({total_neutral} stocks)")
print(f"   Bearish: {pct_bearish:.1f}% ({total_bearish} stocks)")

if pct_bullish > 60:
    market_sentiment = "🟢 VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    market_sentiment = "🟡 BULLISH - Good opportunities"
elif pct_bullish > 20:
    market_sentiment = "🟠 NEUTRAL - Selective opportunities"
else:
    market_sentiment = "🔴 BEARISH - Few opportunities, preserve cash"

print(f"\n📊 Market Sentiment: {market_sentiment}")

# Action items
print("\n✅ ACTION ITEMS FOR THIS WEEK:")
if len(n2_signals) > 0:
    print(f"   1. MONDAY: Sell {len(n2_signals)} N2 positions at market open")
else:
    print("   1. No sells required")

if len(p1_signals) > 0:
    print(f"   2. MONDAY: Enter up to {min(5, len(p1_signals))} new P1 positions")
    print(f"      → Priority: {', '.join(p1_signals.head(5)['Ticker'].tolist())}")
else:
    print("   2. No new buys available - hold cash")

if len(hold_signals) > 0:
    print(f"   3. Monitor {len(hold_signals)} holding positions for state changes")
else:
    print("   3. No positions to monitor")

print("\n" + "=" * 100)

📊 GHB STRATEGY WEEKLY SUMMARY

🟡 BUY Signals (P1):  14 stocks
⚪ HOLD Signals (P2/N1): 6 stocks
🔵 SELL Signals (N2): 5 stocks
📊 Total Scanned: 25/25 stocks

📈 Market Health:
   Bullish: 56.0% (14 stocks)
   Neutral: 24.0% (6 stocks)
   Bearish: 20.0% (5 stocks)

📊 Market Sentiment: 🟡 BULLISH - Good opportunities

✅ ACTION ITEMS FOR THIS WEEK:
   1. MONDAY: Sell 5 N2 positions at market open
   2. MONDAY: Enter up to 5 new P1 positions
      → Priority: MU, MRNA, ALAB, ASML, AMAT
   3. Monitor 6 holding positions for state changes



## 6. Detailed Stock Data

Full dataset for analysis and record-keeping

In [ ]:
# Display full results sorted by state then distance
df_display = df_results.copy()
df_display['State_Order'] = df_display['State'].map({'P1': 1, 'P2': 2, 'N1': 3, 'N2': 4})
df_display = df_display.sort_values(['State_Order', 'Distance_%'], ascending=[True, False])
df_display = df_display.drop('State_Order', axis=1)

print("\n📋 COMPLETE SCAN RESULTS")
print("=" * 100)
print(df_display.to_string(index=False))
print("=" * 100)

## 7. Export Results

Save results to CSV for record-keeping and further analysis

In [ ]:
# Archive old scan results
import os
import shutil
from pathlib import Path

# Create folders if they don't exist
results_dir = Path("../ghb_scanner_results")
archive_dir = results_dir / "archive"
results_dir.mkdir(exist_ok=True)
archive_dir.mkdir(exist_ok=True)

# Move old CSV files to archive
for old_file in results_dir.glob("ghb_strategy_signals_*.csv"):
    try:
        shutil.move(str(old_file), str(archive_dir / old_file.name))
        print(f"📦 Archived: {old_file.name}")
    except Exception as e:
        print(f"⚠️ Could not archive {old_file.name}: {e}")

# Save new results
output_file = results_dir / f"ghb_strategy_signals_{datetime.now().strftime('%Y%m%d')}.csv"
df_results.to_csv(output_file, index=False)

print(f"\n✅ Results saved to: {output_file.name}")
print(f"📁 Location: {output_file.absolute()}")
print(f"📂 Old scans archived to: {archive_dir.name}/")

## GHB Strategy Quick Reference

### Your Optimized Portfolio
**25 stocks:** Your 12 watchlist + 13 top performers  
**Expected:** +514% annual return, ~14 trades/year, 57% win rate

### Entry Rules (BUY)
- State = P1 (Gold)
- Price > 200-day SMA
- Strong momentum (ROC > 5% OR distance > 10%)

### Hold Rules
- **P1 (Gold):** Continue holding, consider adding
- **P2 (Gray):** Hold through consolidation
- **N1 (Gray):** Hold through shallow pullback

### Exit Rules (SELL)
- State = N2 (Blue)
- Price < 200-day SMA
- Weak momentum
- **Execute Monday at open!**

### Expected Performance
- **Annual Return:** +514% (optimized portfolio)
- **Trades Per Year:** ~14 (1-2 per month)
- **Win Rate:** 57%
- **Avg Win:** +64%
- **Avg Loss:** -11%
- **Hold Period:** 8-12 months

### Risk Management
- Max 8-10% per position
- 5-7 concurrent positions typical
- 20-30% cash reserve
- Weekly monitoring only (10-15 minutes)

---
**Next Steps:**
1. Review signals above
2. Execute trades Monday at open
3. Run this notebook again next Friday
4. Track results in portfolio tracker

**Documentation:** See `docs/GHB_STRATEGY_GUIDE.md` for complete strategy details  
**Portfolio List:** See `data/ghb_optimized_portfolio.txt` for your 25 stocks